<a href="https://colab.research.google.com/github/YaCpotato/python/blob/master/CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd drive/'My Drive'/'Colab Notebooks'/

## データセットのロードとプレビュー

In [3]:
import pandas as pd
import io
train = pd.read_csv('./poker-hand-training-true.csv', header=None)
test = pd.read_csv('./poker-hand-testing.csv', header=None)
train.head()
test.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,1,1,13,2,4,2,3,1,12,0
1,3,12,3,2,3,11,4,5,2,5,1
2,1,9,4,6,1,4,3,2,3,9,1
3,1,4,3,13,2,13,2,1,3,6,1
4,3,10,2,7,1,2,2,11,4,9,0


## データ整形：ラベルの抽出と学習、教師データの分割

In [4]:
from sklearn import preprocessing
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import normalize

train_Y = train[:][10]
test_Y = test[:][10]

X_train, X_test, = train_test_split(train,train_size=0.7)
Y_train,Y_test = train_test_split(train_Y,train_size=0.7)
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

X_train.drop(10,axis=1,inplace=True)
X_test.drop(10,axis=1,inplace=True)
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

Using TensorFlow backend.


(17507, 11) (17507,) (7503, 11) (7503,)
(17507, 10) (17507,) (7503, 10) (7503,)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## 学習データからValidation用にデータを取り出す

In [5]:
import numpy as np
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
Y_train = Y_train.astype('float32')
Y_test = Y_test.astype('float32')

X_train, X_val,= train_test_split(X_train,train_size=0.7)
Y_train, Y_val = train_test_split(Y_train,train_size=0.7)
print(X_train.shape,Y_train.shape,X_val.shape,Y_val.shape)

(12254, 10) (12254,) (5253, 10) (5253,)


## CatBoost用のデータにする

In [0]:
!pip install catboost

In [0]:
from catboost import CatBoost
from catboost import Pool

train_pool = Pool(X_train, label=Y_train)
test_pool = Pool(X_test, label=Y_test)

## CatBoostモデルを定義

In [0]:
params = {
        # 学習ラウンド数
        'num_boost_round': 100,
    }
# モデルを学習する
cb = CatBoost(params)

In [13]:
%%time
cb.fit(train_pool)

0:	learn: 0.7704014	total: 3.77ms	remaining: 373ms
1:	learn: 0.7702417	total: 6.6ms	remaining: 324ms
2:	learn: 0.7701424	total: 9.38ms	remaining: 303ms
3:	learn: 0.7699972	total: 12.4ms	remaining: 297ms
4:	learn: 0.7698813	total: 15.3ms	remaining: 290ms
5:	learn: 0.7698481	total: 17.7ms	remaining: 277ms
6:	learn: 0.7697227	total: 20.8ms	remaining: 276ms
7:	learn: 0.7695961	total: 23.6ms	remaining: 271ms
8:	learn: 0.7695154	total: 26.4ms	remaining: 266ms
9:	learn: 0.7693747	total: 29.5ms	remaining: 265ms
10:	learn: 0.7692493	total: 32.2ms	remaining: 260ms
11:	learn: 0.7690949	total: 35ms	remaining: 257ms
12:	learn: 0.7689399	total: 38ms	remaining: 255ms
13:	learn: 0.7688517	total: 41ms	remaining: 252ms
14:	learn: 0.7687316	total: 44.3ms	remaining: 251ms
15:	learn: 0.7686055	total: 47.4ms	remaining: 249ms
16:	learn: 0.7684813	total: 50.2ms	remaining: 245ms
17:	learn: 0.7683381	total: 53.1ms	remaining: 242ms
18:	learn: 0.7682356	total: 55.9ms	remaining: 238ms
19:	learn: 0.7681411	total: 5

評価

In [17]:
# テストデータを予測する
Y_pred = cb.predict(test_pool, prediction_type='Class')

# 精度 (Accuracy) を計算する
from sklearn.metrics import confusion_matrix,classification_report
print (confusion_matrix(Y_test, Y_pred))
print (classification_report(Y_test, Y_pred))

[[   0 3702    0    0    0    0    0    0    0    0]
 [   0 3208    0    0    0    0    0    0    0    0]
 [   0  389    0    0    0    0    0    0    0    0]
 [   0  135    0    0    0    0    0    0    0    0]
 [   0   26    0    0    0    0    0    0    0    0]
 [   0   20    0    0    0    0    0    0    0    0]
 [   0   15    0    0    0    0    0    0    0    0]
 [   0    2    0    0    0    0    0    0    0    0]
 [   0    4    0    0    0    0    0    0    0    0]
 [   0    2    0    0    0    0    0    0    0    0]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      3702
         1.0       0.43      1.00      0.60      3208
         2.0       0.00      0.00      0.00       389
         3.0       0.00      0.00      0.00       135
         4.0       0.00      0.00      0.00        26
         5.0       0.00      0.00      0.00        20
         6.0       0.00      0.00      0.00        15
         7.0       0.00      0.00   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
